In [1]:
import requests
import time
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re

In [2]:
url = 'https://sthaniya.gov.np/gis/website/webdataquery.php?province=0'
page = requests.get(url, timeout=1000, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(page.content, 'html.parser')
txt = 'प्रमुख प्रशासकीय अधिकृत'
phone_regex = r"\b\d{10}\b|\+\d{1,3}-\d{3}-\d{3}-\d{4}"

In [3]:
tbl = soup.find('table')

In [4]:
table = pd.read_html(str(tbl))[0]

In [5]:
table

,क्र.स.,स्थानीय तहको नाम,जिल्ला,स्थानीय तहको प्रकार,प्रदेश,वेवसाईट,कैफियत
0,1,फुङलिङ नगरपालिका,ताप्लेजुङ,नगरपालिका,प्रदेश नम्बर १,phunglingmun.gov.np,तयार
1,2,आठराई त्रिवेणी गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,aathraitribenimun.gov.np,तयार
2,3,सिदिङ्वा गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,sidingbamun.gov.np,तयार
3,4,फक्ताङलुङ गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,phaktanglungmun.gov.np,तयार
4,5,मिक्वाखोला गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,mikwakholamun.gov.np,तयार
...,...,...,...,...,...,...,...
748,749,सुनवल नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,sunwalmun.gov.np,तयार
749,750,सुस्ता गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,sustamun.gov.np,तयार
750,751,पाल्हीनन्दन गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,palhinandanmun.gov.np,तयार
751,752,प्रतापपुर गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,pratappurmun.gov.np,तयार


In [6]:
whole_data = []
unable_to_scrap = []

In [7]:
table.columns

Index(['क्र.स.', 'स्थानीय तहको नाम', 'जिल्ला', 'स्थानीय तहको प्रकार', 'प्रदेश',
       'वेवसाईट', 'कैफियत'],
      dtype='object')

In [8]:
table_data = np.array(table)

In [9]:
table_data

array([[1, 'फुङलिङ नगरपालिका', 'ताप्लेजुङ', ..., 'प्रदेश नम्बर १',
        'phunglingmun.gov.np', 'तयार'],
       [2, 'आठराई त्रिवेणी गाउँपालिका', 'ताप्लेजुङ', ...,
        'प्रदेश नम्बर १', 'aathraitribenimun.gov.np', 'तयार'],
       [3, 'सिदिङ्वा गाउँपालिका', 'ताप्लेजुङ', ..., 'प्रदेश नम्बर १',
        'sidingbamun.gov.np', 'तयार'],
       ...,
       [751, 'पाल्हीनन्दन गाउँपालिका',
        'नवलपरासी (बर्दघाट सुस्ता पश्चिम)', ..., 'लुम्बिनी प्रदेश',
        'palhinandanmun.gov.np', 'तयार'],
       [752, 'प्रतापपुर गाउँपालिका', 'नवलपरासी (बर्दघाट सुस्ता पश्चिम)',
        ..., 'लुम्बिनी प्रदेश', 'pratappurmun.gov.np', 'तयार'],
       [753, 'सरावल गाउँपालिका', 'नवलपरासी (बर्दघाट सुस्ता पश्चिम)', ...,
        'लुम्बिनी प्रदेश', 'sarawalmun.gov.np', 'तयार']], dtype=object)

In [10]:
def get_page(url):
    mun_page = requests.get(url, timeout=5000, headers={'User-Agent': 'Mozilla/5.0'})
    page_soup = BeautifulSoup(mun_page.content, 'html.parser')
    return page_soup

In [11]:
def extractEmail(page_soup):
    page_content = page_soup.text
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', page_content)
    emails = ', '.join(emails)
    return emails

In [12]:
def extractPhone(page_soup):
    phone_regex = r"\b\d{10}\b|\+\d{1,3}-\d{3}-\d{3}-\d{4}"
    phone_number_list = soup.find_all(text=re.compile(phone_regex))
    phone_numbers = ', '.join(phone_number_list)
    return phone_numbers

In [13]:
def check_phone(element):
    if element:
        txt = element.text
        ele = re.findall(phone_regex, txt)
        if len(ele) != 0:
            return True
        else:
            return False
    else:
        return False

In [14]:
def find_numbers(soup):
    element = soup.find(text=re.compile(txt))
    if check_phone(element):
        phone = re.findall(phone_regex, element.text)
        return phone, element.text
    
    if check_phone(element.find_parent()):
        phone = re.findall(phone_regex, element.find_parent().text)
        return phone, element.find_parent().text
    
    if check_phone(element.find_parent().find_parent()):
        phone = re.findall(phone_regex, element.find_parent().find_parent().text)
        return phone, element.find_parent().find_parent().text
    
    if check_phone(element.find_parent().find_parent().find_parent()):
        phone = re.findall(phone_regex, element.find_parent().find_parent().find_parent().text)
        return phone, element.find_parent().find_parent().find_parent().text
    
    if check_phone(element.find_parent().find_parent().find_parent().find_parent()):
        phone = re.findall(phone_regex, element.find_parent().find_parent().find_parent().find_parent().text)
        return phone, element.find_parent().find_parent().find_parent().find_parent().text
    else:
        return '', ''

In [15]:
counter = 1
for td in table_data:
    print(counter-1)
    mun_url = 'http://'+td[5]
    print('Scrapping', 'http://'+td[5])
    try:
        soup = get_page(mun_url)
        phone_number, cao_details = find_numbers(soup)
        phone_numbers = extractPhone(soup)
        emails = extractEmail(soup)
        whole_data.append({
            'sn': td[0],
            'municipality': td[1],
            'District': td[2],
            'Local_level_Type': td[3],
            'Province': td[4],
            'website': mun_url,
            'cao_name_phone': cao_details,
            'cao_phone_number': str(phone_number),
            'emails': emails,
            'phone_numbers': phone_numbers
        })
        print(str(phone_number))
        print(cao_details)
        print(phone_numbers)
        print(emails)
        print('Completed\n')
    except:
        unable_to_scrap.append(int(td[0])-1)
        print('unabel to scrap')
    counter = counter + 1

0
Scrapping http://phunglingmun.gov.np
['9852622111']

  
 जय नारायण खनाल 
 जय नारायण खनाल (प्रमुख प्रशासकीय अधिकृत) 
 CAO@phunglingmun.gov.np 
 9852622111  
9842660480, ९८४२६९११५१/9852660208, 9852622111
mayor@phunglingmun.gov.np, deputy.mayor@phunglingmun.gov.np, CAO@phunglingmun.gov.np, info@phunglingmun.gov.np, phunglingmun@gmail.com
Completed

1
Scrapping http://aathraitribenimun.gov.np
['9852660960']

  
 भक्त बहादुर भट्टराई 
 प्रमुख प्रशासकीय अधिकृत 
 फोन नं :  9852660960  
9852660960, ९८५२६६०९५८, ९८४२६५९१०४, 9852660960, प्रमुख प्रशासकीय अधिकृत :- ९८५२६६०९६० 
aathrai.tribenimun@gmail.com, aathrai.tribenimun@gmail.com
Completed

2
Scrapping http://sidingbamun.gov.np
unabel to scrap
3
Scrapping http://phaktanglungmun.gov.np
['९८५२६६०४९१']

  
 नवीन तेम्वे 
 प्रमुख प्रशासकीय अधिकृत 
 मोबाईल नं. :  ९८५२६६०४९१  
९८५१२०४८९४, ९८४४६४८२५४, ९८५२६६०४९१, ९८५२६६०१७८, ९८५२६६०३०१, ९८५२६६०३१०, ९८५२६६०३०२, ९८५२६६०४८५
phaktanglungmun@gmail.com
Completed

4
Scrapping http://mikwakholamun.gov.np


स



9852685445, 
९८५२६८३०६० (अध्यक्ष), 
९८६७०६९०७७ (उपाध्यक्ष), 
९८५२६८५४४५ (प्रमुख प्रशासकीय अधिकृत), 
९८५११६६४९६ (सुचना अधिकारी)
mangsebung.ruralmun@gmail.com, mangsebung.ruralmun@gmail.com, info@mansebungmun.gov.np, ito.mansebungmunilam@gmail.com
Completed

25
Scrapping http://rongmun.gov.np
['9852642020']

  
 अनिल तिखत्री 
 प्रमुख प्रशासकीय अधिकृत 
 Phone Number:  9852642020  
9852678732, 9844631517, 9852642020, ९८४१७००२४८, 9842643724, 9843151567
rongruralmun@gmail.com
Completed

26
Scrapping http://sandakpurmun.gov.np
unabel to scrap
27
Scrapping http://mechinagarmun.gov.np
['9852629111']



                               प्रमुख प्रशासकीय अधिकृत 		


                                                                                                                                                                                                    

टेक कुमार रेग्मी  
ito.mechinagarmun@gmail.com
 9852629111, 023562437


023562505/9840995657 (Ashok Bhetwal, Information Technology Officer



सम्पर्क नं : ९८१९०१२३६६ 
info@gramthanmun.gov.np, ito.gramthanmun@gmail.com
Completed

54
Scrapping http://jahadamun.gov.np
['9820730031']

  
 सुरेन्द्र कुमार यादव 
 प्रमुख प्रशासकीय अधिकृत 
 Phone:  9820730031  
9852044821, 9802709990, 9802719990, 9820730031, सम्पर्क नं. 9852081008
jahadagaupalika@gmail.com, ito.jahadamun@gmail.com, jitendrapdshah@gmail.com, hannasoren@gmail.com, jahadagaupalika@gmail.com, ito.jahadamun@gmail.com, suchanaadhikari@jahadamun.gov.np
Completed

55
Scrapping http://kanepokharimun.gov.np
['९८५२०३८४०६']

  
 हिराकुमारी यादव 
 प्रमुख प्रशासकीय अधिकृत 
 ९८५२०३८४०६  
९८४२४३६३०४, ९८५२०६०५६२, ९८५२०३८४०६, ९८५२०३८४०९, ९८५२०२२०३३, ९८४०८७८१९९,९८०२७६१५१८, ९८०८७१५९९९
kanepokharirmun@gmail.com, kanepokharirmun@gmail.com, suchanaadhikari@kanepokharimun.gov.np
Completed

56
Scrapping http://kataharimun.gov.np
['9852023528']

  
 मनोज कुमार साह  
 प्रमुख प्रशासकीय अधिकृत 
 9852023528  
9852029427, 9807351703, 9852023528, 9816398821, : 9746899249,  (Jeewan Bikas PHC: 980

['9849670377', '9852086531']

                    
                      राजिव अधिकारी
                   9849670377 / 9852086531
              प्रमुख प्रशासकीय अधिकृत
                                                                              


9852099753,9852086530,                    9849670377 / 9852086531, 9849842122, 9741406617, फाेन नं. 9852099753 (गा.पा. प्रमुख)/ 9852086530(प्रशासकिय अधिकृत)
bhotkholamun@gmail.com, bhotkholamun@gmail.com
Completed

90
Scrapping http://chichilamun.gov.np
['9843709889']

  
 सन्तोष राई 
 प्रमुख प्रशासकीय अधिकृत 
 9843709889 
 rai23.ksantosh@gmail.com   
9860354626, 9862128113, 9742482111, 9843709889, 9852099683 ( Office Hours ), श्री शान्तिराम गुरुङ (9860717252), श्री ख्यामराज राई (9842482028), मोबाइल नं: ९८४३७०९८८९
chichilagaunpalika@gmail.com, chichilagaunpalika@gmail.com, chichilagaunpalika@gmail.com, rai23.ksantosh@gmail.com, chichilagaunpalika@gmail.com, info@chichilamun.gov.np, suchanaadhikari@chichilamun.gov.np, info@chichilamun.gov.np,

unabel to scrap
111
Scrapping http://siddhicharanmun.gov.np


९८४१३१२३४९, ९८६२६०५१८८, ९८५२८२३१११, ९८५२८४३३३३, Viber No. : 9851032715 
mayor@siddhicharanmun.gov.np, basnetkedar138@gmail.com, siddhicharanmunicipality@gmail.com, acharyasusma66@gmail.com, siddhicharanmunicipality@gmail.com, padam@siddhicharanmun.gov.np
Completed

112
Scrapping http://khijidembamun.gov.np



सम्पर्क न: ९८४९६७००८९ , कार्यालय  सम्पर्क  नं  : +९७७-९७४८६४२७८९ , गाउँपालिका अध्यक्ष सम्पर्क  नं  : ९८५११३०८०१, इमरजेन्सि एम्बुलेन्स सम्पर्क न‌ः ९७६१६१४८८१(वाङ्डा शेर्पा), मोबाइल नं  : ९८४९६७००८९
info@khijidembamun.gov.np, khijeedembarm2073@gmail.com, suchana.adhikari@khijidembamun.gov.np, info@khijidembamun.gov.np, khijeedembarm2073@gmail.com, ito.khijeedembarmun@gmail.com
Completed

113
Scrapping http://champadevimun.gov.np
['९८५२८४४४४८']

  
 प्रेम प्रसाद सुवेदि 
 नि‍ प्रमुख प्रशासकीय अधिकृत 
 ito.champadevimun@gmail.com 
 ९८५२८४४४४८  

          ९८५१०६१६८४,            ९८४३१८१९८३        , ९८५२८४४४४८, ९८५२८४४४४९
ito.

...

unabel to scrap
619
Scrapping http://mudkechulamun.gov.np


9858390100, सम्पर्क नं :9854044310, सम्पर्क नं:9858390100,9854044310
इमेल :info@mudkechulamun.gov.np,
ito.mudkechulamun@gmail.com
ito.mudkechulamun@gmail.com, info@mudkechulamun.gov.np, suchanaadhikari@mudkechulamun.gov.np, info@mudkechulamun.gov.np, ito.mudkechulamun@gmail.com
Completed

620
Scrapping http://kaikemun.gov.np
['9858390164', '9841938164']

  
 सम्झना पराजुली 
 प्रमुख प्रशासकीय अधिकृत 
 9858390164/9841938164 
 samjhanaparajuli123@gmail.com	  
9868055466, 9849364077, 9848387996, 9858390164/9841938164, 9868055466, ९८६३२८१०९०, मोबाईल नम्बरः 9858028133, फोन नम्बर: (9868055466)
ito.kaikemun@gmail.com, samjhanaparajuli123@gmail.com, smydream77@gmail.com, dangibirendra94@gmail.com, ito.kaikemun@gmail.com, smydream77@gmail.com, suchanaadhikari@kaikemun.gov.np
Completed

621
Scrapping http://chharkatangsongmun.gov.np
unabel to scrap
622
Scrapping http://chandannathmun.gov.np
unabel to scrap
623
Scrapping http://kankasunda



९८५१०७९४२०, ९८५१०७९४२०, ९८४८१५७२१८, ९८४८१८६५३५, ९८५११९९२९५, 9848138204, 9858322074, 9858086200, 9848318302, 
          9848336035        , 
          9843642099        , 
          9858322687   ,   9848369304        , 
          9858321084        , 
          9843931500        , 
          9844872636        , 
नाम :  सुरेन्द्र फडेरा
पद : सूचना तथा संचार प्रविधि अधिकृत
इमेल :suchanaadhikari@adanchulimun.gov.np
फोन नं. : ९८४८३३६०३५
फोटो 

, 
अदानचुली गाउँपालिका
माेवाइल नं.- ९८५१०७९४२०,९८४८३३६०३५
इमेल - info@adanchulimun.gov.np , ito.adanchulimun@gmail.com  ito@adanchulimun.gov.np
वेवसाइट- www.adanchulimun.gov.np
page - Adanchuli
Facebook - https://www.facebook.com/adanchuli.municipality


info@adanchulimun.gov.np, ito.adanchulimun@gmail.com, singhmohanbikram49@gmail.com, karnarokaya37@gmail.com, akamahatara@gmail.com, nandmahatara@gmail.com, phaderadilbahadur@gmail.com, manarishipandeyjaishi2019@gmail.com, kamal.chhatyal@gmail.com, surendraphadera153@gmail.com, rdsbista@gmail.com, datt

['9860659938']

  
 प्रकाश दुंगाना  
 प्रमुख प्रशासकीय अधिकृत 
 9860659938  
9849549412, 9848621122, ९८४८६९०६१९, 9860659938, ९८४९७११६२१, फोन:  9748854444
info@bannigadhijaygadhmun.gov.np, log.bannigadhi@gmail.com, thakullatilak48@gmail.com
Completed

678
Scrapping http://ramaroshanmun.gov.np
unabel to scrap
679
Scrapping http://dhakarimun.gov.np
['9864811102']

  
 लक्ष्मण शाह 
 नि. प्रमुख प्रशासकीय अधिकृत 
 sahlaxman273@gmail.com  
 9864811102  
9864811311, 9860371282, 9840658075, 9864811102, 9864811311
ito.dhakarimun@gmail.com, netrakaladevishahi@gmail.com, sahlaxman273@gmail.com, suchanaadhikari@dhakarimun.gov.np, ito.dhakarimun@gmail.com, ito.dhakarimun@gmail.com, info@dhakarimun.gov.np, suchanaadhikari@dhakarimun.gov.np
Completed

680
Scrapping http://turmakhadmun.gov.np
unabel to scrap
681
Scrapping http://dipayalsilgadhimun.gov.np
unabel to scrap
682
Scrapping http://shikharmun.gov.np
unabel to scrap
683
Scrapping http://purbichaukimun.gov.np
unabel to scrap
684
Scrapping http:/

['९८५१२०४१४३']

  
 टिकेन्द्र राज भट्ट 
 प्रमुख प्रशासकीय अधिकृत 
 ९८५१२०४१४३  
9848723185, 9848423148, 9858750464, ९८५१२०४१४३, 9858756594, ९८६८५७८६२८, 
सूचना अधिकारी: मोहन राज फुलारा (9868578628)
info@shuklaphantamun.gov.np, info@shuklaphantamun.gov.np, suchana.adhikari@shuklaphantamun.gov.np
Completed

708
Scrapping http://belaurimun.gov.np
['9868849245']

  
 तारा दत्त अवस्थी 
 प्रमुख प्रशासकीय अधिकृत 
  
 9868849245  
9858751287; 9809454671, 9868551958; 9812789383, 9868849245, 9800626978
belaurimunicipality@gmail.com, info@belaurimun.gov.np
Completed

709
Scrapping http://krishnapurmun.gov.np
['9858732111']

  
 श्री शिव सिंह भण्डारी  
 प्रमुख प्रशासकीय अधिकृत 
 9858732111  
9858750621, 9858784561, 9858732111, ९८५८७५०६८५, कार्यलय प्रमुख -९८५८७३२१११, 
सूचना अधिकारी - ९८५८७५०६८५, 
कृष्णपुर नगरपालिका एम्बुलेन्स सेवा - ९८६५५७६००७
ito.krishnapurmun@gmail.com, mayor@krishnapurmun.gov.np, droppatirana2046@gmail.com, info@krishnapurmun.gov.np, ito.krishnapurmun@gmail.com
Completed

710
Scr

['9858750342']

  
 रघुनाथ अवस्थी 
 प्रमुख प्रशासकीय अधिकृत 
 9858750342  
9858750342, ९८५८७३६९९३, ९७६९८७३१७०, 9858750342, ९७६९८७३१८०, ९७६९८७३२०४, फाेन नं. ९७६९८७३१८०, फोन नं. ९८५८७५०३४२
drmbaitadi@gmail.com, ward5@dilasainimun.gov.np, suchanaadhikari@dilasainimun.gov.np, info@dilasainimun.gov.np, drmbaitadi@gmail.com
Completed

729
Scrapping http://mahakalimundarchula.gov.np
unabel to scrap
730
Scrapping http://shailyashikharmun.gov.np



shailyashikhar.gov@gmail.com, ito.shailyashikharmun@gmail.com
Completed

731
Scrapping http://malikarjunmun.gov.np


एम्बुलेन्स सम्पर्क व्यक्तिः-९८५१२१५८६९,चालकः-९८४८९९९७७१,९८६८५१२६०९(पस्ती),९८४८७६३०२१(उकू), ९८४८८७५३४१, ९८४८८१३८४०, ९८५८७८४५१३, ९८६८४६१०९६
mrmdarchula@gmail.com, info@malikarjunmun.gov.np, hiradhami.ddo@gmail.com, pankajdhami633@gmail.com, cao.mrmdarchula@gmail.com, lokendradhami2046@gmail.com, mrmdarchula@gmail.com, info@malikarjunmun.gov.np
Completed

732
Scrapping http://apihimalmun.gov.np
unabel to scrap
733
Scrapping http://duhunmu

In [16]:
whole_data

[{'sn': 1,
  'municipality': 'फुङलिङ नगरपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'नगरपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://phunglingmun.gov.np',
  'cao_name_phone': '\n  \n जय नारायण खनाल \n जय नारायण खनाल (प्रमुख प्रशासकीय अधिकृत) \n CAO@phunglingmun.gov.np \n 9852622111  ',
  'cao_phone_number': "['9852622111']",
  'emails': 'mayor@phunglingmun.gov.np, deputy.mayor@phunglingmun.gov.np, CAO@phunglingmun.gov.np, info@phunglingmun.gov.np, phunglingmun@gmail.com',
  'phone_numbers': '9842660480, ९८४२६९११५१/9852660208, 9852622111'},
 {'sn': 2,
  'municipality': 'आठराई त्रिवेणी गाउँपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'गाउँपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://aathraitribenimun.gov.np',
  'cao_name_phone': '\n  \n भक्त बहादुर भट्टराई \n प्रमुख प्रशासकीय अधिकृत \n फोन नं :  9852660960  ',
  'cao_phone_number': "['9852660960']",
  'emails': 'aathrai.tribenimun@gmail.com, aathrai.tribenimun@gmail.com',
  'ph

In [17]:
email_df = pd.DataFrame(whole_data)

In [18]:
email_df

,sn,municipality,District,Local_level_Type,Province,website,cao_name_phone,cao_phone_number,emails,phone_numbers
0,1,फुङलिङ नगरपालिका,ताप्लेजुङ,नगरपालिका,प्रदेश नम्बर १,http://phunglingmun.gov.np,\n \n जय नारायण खनाल \n जय नारायण खनाल (प्रमु...,['9852622111'],"mayor@phunglingmun.gov.np, deputy.mayor@phungl...","9842660480, ९८४२६९११५१/9852660208, 9852622111"
1,2,आठराई त्रिवेणी गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://aathraitribenimun.gov.np,\n \n भक्त बहादुर भट्टराई \n प्रमुख प्रशासकीय...,['9852660960'],"aathrai.tribenimun@gmail.com, aathrai.tribenim...","9852660960, ९८५२६६०९५८, ९८४२६५९१०४, 9852660960..."
2,4,फक्ताङलुङ गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://phaktanglungmun.gov.np,\n \n नवीन तेम्वे \n प्रमुख प्रशासकीय अधिकृत ...,['९८५२६६०४९१'],phaktanglungmun@gmail.com,"९८५१२०४८९४, ९८४४६४८२५४, ९८५२६६०४९१, ९८५२६६०१७८..."
3,5,मिक्वाखोला गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://mikwakholamun.gov.np,,,"suchanaadhikari@mikwakholamun.gov.np, mikwakho...","सम्पर्क नं. 9844632687, गाउँपालिका अध्यक्ष सम..."
4,9,सिरीजङ्घा गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://sirijanghamun.gov.np,,,"sirijangharuralmunicipality@gmail.com, lalkris...","९८५२६६०३३१, ९८४२७७७८८६, \nसम्पर्क नं.: ९८५२६६०..."
...,...,...,...,...,...,...,...,...,...,...
580,748,रामग्राम नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,http://ramgrammun.gov.np,\n \n मोहन अर्याल \n प्रमुख प्रशासकीय अधिकृत ...,['9857628111'],municipalityramgram@gmail.com,"9857628111 , ९८५७०४७५७२, , Mobile No. 9866154562"
581,749,सुनवल नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,http://sunwalmun.gov.np,\n \n चेतनाथ गिरी \n प्रमुख प्रशासकीय अधिकृत...,['९८५७६३८१११'],"sunwalmunicipality@gmail.com, suchanaadhikari@...","९८५७०४८१०२, ९८५७०४८१०३, 9847287481, ९८५७६३८..."
582,750,सुस्ता गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://sustamun.gov.np,\n \n हुमलाल भुसाल \n प्रमुख प्रशासकीय अधिकृत...,['9857082320'],"sustagaunpalika@gmail.com, sustagaunpalika@gma...","9857080160, 9847064276, 9857082320, 9847152519..."
583,751,पाल्हीनन्दन गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://palhinandanmun.gov.np,,,"ito.palhinandanmun@gmail.com, info@palhinandan...","\n ९८५७०२४२९६ , \n 98..."


In [19]:
email_df.to_excel("municipality_contact_info.xlsx")

In [20]:
unable_to_scrap

[2,
 5,
 6,
 7,
 13,
 26,
 33,
 34,
 39,
 44,
 45,
 46,
 51,
 60,
 65,
 67,
 68,
 69,
 71,
 77,
 78,
 79,
 84,
 99,
 100,
 110,
 127,
 135,
 139,
 145,
 157,
 162,
 166,
 173,
 175,
 177,
 178,
 183,
 184,
 186,
 192,
 194,
 195,
 198,
 199,
 206,
 209,
 210,
 213,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 231,
 236,
 256,
 257,
 272,
 275,
 279,
 286,
 287,
 291,
 295,
 296,
 297,
 305,
 327,
 341,
 345,
 346,
 347,
 354,
 356,
 357,
 364,
 365,
 369,
 370,
 373,
 375,
 377,
 379,
 382,
 393,
 402,
 413,
 421,
 428,
 431,
 458,
 459,
 460,
 462,
 466,
 470,
 472,
 486,
 488,
 496,
 500,
 501,
 504,
 511,
 518,
 521,
 540,
 542,
 552,
 553,
 560,
 577,
 581,
 592,
 593,
 601,
 606,
 614,
 618,
 621,
 622,
 623,
 624,
 626,
 631,
 634,
 639,
 641,
 642,
 651,
 652,
 654,
 655,
 656,
 657,
 659,
 661,
 663,
 664,
 666,
 668,
 670,
 671,
 678,
 680,
 681,
 682,
 683,
 687,
 688,
 689,
 698,
 702,
 703,
 705,
 718,
 719,
 723,
 724,
 729,
 732,
 737,
 746,
 751]

In [21]:
email_df

,sn,municipality,District,Local_level_Type,Province,website,cao_name_phone,cao_phone_number,emails,phone_numbers
0,1,फुङलिङ नगरपालिका,ताप्लेजुङ,नगरपालिका,प्रदेश नम्बर १,http://phunglingmun.gov.np,\n \n जय नारायण खनाल \n जय नारायण खनाल (प्रमु...,['9852622111'],"mayor@phunglingmun.gov.np, deputy.mayor@phungl...","9842660480, ९८४२६९११५१/9852660208, 9852622111"
1,2,आठराई त्रिवेणी गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://aathraitribenimun.gov.np,\n \n भक्त बहादुर भट्टराई \n प्रमुख प्रशासकीय...,['9852660960'],"aathrai.tribenimun@gmail.com, aathrai.tribenim...","9852660960, ९८५२६६०९५८, ९८४२६५९१०४, 9852660960..."
2,4,फक्ताङलुङ गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://phaktanglungmun.gov.np,\n \n नवीन तेम्वे \n प्रमुख प्रशासकीय अधिकृत ...,['९८५२६६०४९१'],phaktanglungmun@gmail.com,"९८५१२०४८९४, ९८४४६४८२५४, ९८५२६६०४९१, ९८५२६६०१७८..."
3,5,मिक्वाखोला गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://mikwakholamun.gov.np,,,"suchanaadhikari@mikwakholamun.gov.np, mikwakho...","सम्पर्क नं. 9844632687, गाउँपालिका अध्यक्ष सम..."
4,9,सिरीजङ्घा गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://sirijanghamun.gov.np,,,"sirijangharuralmunicipality@gmail.com, lalkris...","९८५२६६०३३१, ९८४२७७७८८६, \nसम्पर्क नं.: ९८५२६६०..."
...,...,...,...,...,...,...,...,...,...,...
580,748,रामग्राम नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,http://ramgrammun.gov.np,\n \n मोहन अर्याल \n प्रमुख प्रशासकीय अधिकृत ...,['9857628111'],municipalityramgram@gmail.com,"9857628111 , ९८५७०४७५७२, , Mobile No. 9866154562"
581,749,सुनवल नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,http://sunwalmun.gov.np,\n \n चेतनाथ गिरी \n प्रमुख प्रशासकीय अधिकृत...,['९८५७६३८१११'],"sunwalmunicipality@gmail.com, suchanaadhikari@...","९८५७०४८१०२, ९८५७०४८१०३, 9847287481, ९८५७६३८..."
582,750,सुस्ता गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://sustamun.gov.np,\n \n हुमलाल भुसाल \n प्रमुख प्रशासकीय अधिकृत...,['9857082320'],"sustagaunpalika@gmail.com, sustagaunpalika@gma...","9857080160, 9847064276, 9857082320, 9847152519..."
583,751,पाल्हीनन्दन गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://palhinandanmun.gov.np,,,"ito.palhinandanmun@gmail.com, info@palhinandan...","\n ९८५७०२४२९६ , \n 98..."


In [22]:
# url = 'https://phunglingmun.gov.np/'
# url = 'https://aathraitribenimun.gov.np/'
# url = 'https://phunglingmun.gov.np/'


In [23]:
# soup = get_page(url)

In [24]:
# ph, text_detail = find_numbers(soup)
# print(ph, text_detail)

In [25]:
# target_element.find_parent().find_parent().find_parent().text

In [26]:
# re.findall(phone_regex, target_element.find_parent().find_parent().find_parent().text)